In [6]:
import pandas as pd
import plotly.express as px
from ipywidgets import Dropdown, VBox, HBox, Layout, Output
from IPython.display import display, Markdown, HTML

# Hide code inputs (Voila only)
display(HTML("<style>.jp-Cell-inputWrapper {display: none;}</style>"))

# Load and clean data
df = pd.read_excel("C:\\Users\\Admin\\Downloads\\Trainer Information and Batch Monitoring Data.xlsx")
df.columns = df.columns.str.strip()

# Clean numeric columns
numeric_cols = [
    'Total Trainees Completed in batch 1', 'Males Completed in batch 1', 'Females Completed in batch 1',
    'Employed - Males from batch 1', 'Employed - Females from batch 1',
    'Unemployed - Males from batch 1', 'Unemployed - Females from batch 1',
    'Number of new Trainees', '2nd Batch - Males', '2nd Batch - Females'
]
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

# Widgets
location_widget = Dropdown(
    options=['All'] + sorted(df['Location'].dropna().unique().tolist()),
    value='All',
    description='Location:',
    layout=Layout(width='50%')
)

industry_widget = Dropdown(
    options=['All'],
    value='All',
    description='Industry:',
    layout=Layout(width='50%')
)

# Output container
output = Output()

# Filter DataFrame
def filter_df(location, industry):
    filtered = df.copy()
    if location != 'All':
        filtered = filtered[filtered['Location'] == location]
    if industry != 'All':
        filtered = filtered[filtered['Industry'] == industry]
    return filtered

# KPI text
def create_kpis(filtered):
    total_trainees = filtered['Total Trainees Completed in batch 1'].sum()
    males = filtered['Males Completed in batch 1'].sum()
    females = filtered['Females Completed in batch 1'].sum()
    employed = filtered['Employed - Males from batch 1'].sum() + filtered['Employed - Females from batch 1'].sum()
    unemployed = filtered['Unemployed - Males from batch 1'].sum() + filtered['Unemployed - Females from batch 1'].sum()

    return f"""
### 📊 KPIs:
- **Total Trainees Completed (Batch 1):** {int(total_trainees)}  
- 👨 **Males Completed:** {int(males)}  
- 👩 **Females Completed:** {int(females)}  
- ✅ **Employed:** {int(employed)}  
- ❌ **Unemployed:** {int(unemployed)}
"""

# Charts
def plot_charts(filtered):
    bar_fig = px.bar(
        filtered,
        x='Industry',
        y='Total Trainees Completed in batch 1',
        title='Total Trainees Completed in Batch 1 by Industry',
        text='Total Trainees Completed in batch 1'
    )
    bar_fig.update_traces(textposition='outside')

    gender_counts = {
        'Males': filtered['Males Completed in batch 1'].sum(),
        'Females': filtered['Females Completed in batch 1'].sum()
    }
    pie_fig = px.pie(
        names=gender_counts.keys(),
        values=gender_counts.values(),
        title='Gender Distribution of Trainees Completed Batch 1'
    )

    return bar_fig, pie_fig

# Update industry options dynamically
def update_industry_options(change):
    selected_location = change['new']
    if selected_location == 'All':
        industries = sorted(df['Industry'].dropna().unique().tolist())
    else:
        industries = sorted(df[df['Location'] == selected_location]['Industry'].dropna().unique().tolist())

    industry_widget.options = ['All'] + industries
    industry_widget.value = 'All'

# Dashboard update logic
def update_dashboard(change=None):
    with output:
        output.clear_output(wait=True)
        filtered = filter_df(location_widget.value, industry_widget.value)
        display(Markdown(create_kpis(filtered)))
        bar_fig, pie_fig = plot_charts(filtered)
        bar_fig.show()
        pie_fig.show()

# Observers
location_widget.observe(update_industry_options, names='value')
location_widget.observe(update_dashboard, names='value')
industry_widget.observe(update_dashboard, names='value')

# Layout
ui = VBox([HBox([location_widget, industry_widget]), output])

# Display interface
display(ui)

# Initial load
update_industry_options({'new': location_widget.value})
update_dashboard()
